핸드폰 사용에 대한 일반적 패턴과 개인적 패턴 구분하기

개인적 패턴을 활용하여 personalized channel recommendation 모델 만들기 

## **🔸 DATA OVERVIEW**
- 스마트폰 앱/웹 로그 데이터셋
- Android 6.0.0 이상을 대상으로 UsageStat API를 활용해 수집된 앱 사용 내역
- 31명의 피험자를 대상으로 2018.10월-11월 중 약 3주간 수집됨, 약 158만 건
- json형태로 타임스탬프, 기기, 패키지명, 사용어플명, 타입(포그라운드, 백그라운드) 등에 대한 내용을 담고 있음

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
cd /content/drive/MyDrive/토스

/content/drive/MyDrive/토스


In [8]:
ls

스마트폰_로그_데이터셋2.json  토스분석.csv  스마트폰_로그_데이터셋.json


In [9]:
import json
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(["seaborn-whitegrid"])
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


In [ ]:
f1 = open("스마트폰_로그_데이터셋.json", encoding="UTF-8")
f2 = open("스마트폰_로그_데이터셋2.json", encoding="UTF-8")

json1 = json.loads(f1.read())
json2 = json.loads(f2.read())

In [ ]:
json1

[{'id': 4,
  'is_system_app': 0,
  'is_updated_system_app': 0,
  'name': 'BAND',
  'package_name': 'com.nhn.android.band',
  'source_info': 'samsung-SM-G928S-7.0',
  'source_type': 'PHONE',
  'subject_id': '2099efc17d719274924b8c81543c7972',
  'timestamp': 1540790989686,
  'type': 'USER_INTERACTION',
  'utcoffset': 9},
 {'id': 7,
  'is_system_app': 0,
  'is_updated_system_app': 0,
  'name': 'BAND',
  'package_name': 'com.nhn.android.band',
  'source_info': 'samsung-SM-G928S-7.0',
  'source_type': 'PHONE',
  'subject_id': '2099efc17d719274924b8c81543c7972',
  'timestamp': 1540790994228,
  'type': 'MOVE_TO_FOREGROUND',
  'utcoffset': 9},
 {'id': 10,
  'is_system_app': 0,
  'is_updated_system_app': 0,
  'name': 'BAND',
  'package_name': 'com.nhn.android.band',
  'source_info': 'samsung-SM-G928S-7.0',
  'source_type': 'PHONE',
  'subject_id': '2099efc17d719274924b8c81543c7972',
  'timestamp': 1540790994420,
  'type': 'MOVE_TO_BACKGROUND',
  'utcoffset': 9},
 {'id': 13,
  'is_system_app': 0

In [151]:
df1 = pd.DataFrame(json1)
df1= df1.drop(columns=["id"])
df1.head(2)

,timestamp,utcoffset,subject_id,source_type,source_info,package_name,name,is_system_app,is_updated_system_app,type
0,1540790989686,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,USER_INTERACTION
1,1540790994228,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND


In [152]:
df1['timestamp'] = df1['timestamp'].apply(lambda time: datetime.datetime.fromtimestamp(int(time)//1000))

In [153]:
df1[:30]

,timestamp,utcoffset,subject_id,source_type,source_info,package_name,name,is_system_app,is_updated_system_app,type
0,2018-10-29 05:29:49,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,USER_INTERACTION
1,2018-10-29 05:29:54,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND
2,2018-10-29 05:29:54,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_BACKGROUND
3,2018-10-29 05:29:54,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND
4,2018-10-29 05:30:01,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_BACKGROUND
5,2018-10-29 05:30:01,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND
6,2018-10-29 05:30:07,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_BACKGROUND
7,2018-10-29 05:30:07,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND
8,2018-10-29 05:30:22,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_BACKGROUND
9,2018-10-29 05:30:22,9,2099efc17d719274924b8c81543c7972,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND


In [154]:
# subject_id는 수집 대상 인원을 암호화된 것으로 식별하기 쉽게 0~32로 라벨링을 진행하였습니다. 
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df1["subject_id"] = encoder.fit_transform(df1["subject_id"])
df1["subject_id"].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])

In [155]:
# 굳이 정렬이 되어 있는 녀석을 따로 정렬할 필요가 없을 것 같다! 오히려 잘못정렬해서 앱 사용시간에 혼돈을 줄 수 있을 거 같다 
# df1 = df1.sort_values(by=["subject_id", "timestamp"])

In [156]:
# null 값 확인, unique확인, 데이터 타입 확인
checking = df1.nunique().to_frame(name="unique")
checking["null"] =df1.isnull().sum()
checking["dtypes"] = df1.dtypes
checking.T

,timestamp,utcoffset,subject_id,source_type,source_info,package_name,name,is_system_app,is_updated_system_app,type
unique,633587,2,33,1,27,864,833,2,2,6
null,0,0,0,0,0,0,0,0,0,0
dtypes,datetime64[ns],int64,int64,object,object,object,object,int64,int64,object


timestamp : 핸드폰에서 어플 사용할 때마다 시간이 측정됨  
utc offset : 시간 기준으로 한국에서 사용했는지 해외에서 사용했는지를 나타내는 지표  
subject_id : 실험에 참여한 인원들  
source_type : 핸드폰 앱 사용 기록이므로 핸드폰 1종류만 기재됨    
source_info	: 핸드폰에 대한 상세정보(모델명, 버전)    
package_name : 핸드폰에서 사용한 어플에 대한 경로값 	    
name : 사용한 어플 이름   
is_system_app	: 핸드폰 내에 설치되어 있는 앱  
is_updated_system_app	: 플레이 스토어 등을 통해서 추가적으로 설치된 앱  
type : 사용자가 어떤 행동을 하였는지 기록(대분류로 분류되어 있으나, 사용한 앱에 따라 별도의 분류가 필요할 것으로 사료됨)  

In [157]:
# 해외에서 사용한 이력도 645개로 관찰된다 
df1[["type"]].value_counts().to_frame()

,0
type,
MOVE_TO_FOREGROUND,739388
MOVE_TO_BACKGROUND,737798
USER_INTERACTION,93238
CONFIGURATION_CHANGE,17218
SHORTCUT_INVOCATION,498
NONE,198


In [158]:
# 해외에서 사용한 이력도 645개로 관찰된다 
df1[["utcoffset"]].value_counts().to_frame().T

utcoffset,9,0
0,1587693,645


In [159]:
# 2, 18번 사용자가 압도적으로 많은 사용량이 관측됨 
df1[["subject_id"]].value_counts()

subject_id
2             130600
18            129091
27             76989
1              76904
25             76860
15             72934
17             67326
20             63364
21             57593
23             57349
9              57133
6              51969
3              51539
19             51421
12             50692
14             47764
22             43688
0              41334
4              39344
32             34508
8              33892
26             30124
7              29861
13             27382
11             26971
29             26214
31             25739
24             24394
28             24304
5              23242
10             20516
16             16615
30               682
dtype: int64

In [160]:
# 핸드폰 취향에 따라서 사용자의 패턴이 달라질 수 있으므로, 큰 카테고리 ( 삼성, LG, google 등)으로 범위화 하고자 함 
df1[["source_info"]].value_counts()

source_info                           
samsung-SM-G950N-8.0.0                    196556
samsung-SM-G930K-8.0.0                    141533
Sony-G8141-8.0.0                          130600
samsung-SM-G965N-8.0.0                    130025
LGE-LG-F800K-8.0.0                        129091
samsung-SM-N950N-8.0.0                    118712
samsung-SM-G930S-8.0.0                     88830
LGE-LGM-V300K-8.0.0                        72934
samsung-SM-A710S-7.0                       67326
samsung-SM-G960U1-8.0.0                    57593
samsung-SM-N920S-7.0                       50692
samsung-SM-N910K-6.0.1                     47764
samsung-SM-G960N-8.0.0                     41334
samsung-SM-G935S-8.0.0                     39344
samsung-SM-N960N-8.1.0                     38846
samsung-SM-N935S-8.0.0                     34508
samsung-SM-N920K-7.0                       33892
samsung-SM-G955N-8.0.0                     30124
LGE-LGM-G600S-8.0.0                        26971
LGE-LG-F700L-8.0.0            

In [161]:
# 해당 데이터는 본 분석에 크게 의미가 있을 것으로 사료되지 않음 
print("총 개수 : ", len(df1[["package_name"]].value_counts()))
print("---")
print("카테고리")
print(df1[["package_name"]].value_counts())

총 개수 :  864
---
카테고리
package_name                 
com.kakao.talk                   427579
com.sec.android.app.launcher     164901
com.twitter.android              151947
kaist.iclab.abc                   50838
com.cashwalk.cashwalk             45289
                                  ...  
com.samsung.android.easysetup         1
com.brainbow.peak.app                 1
com.belugaedu.amgigorae               1
com.photofunia.android                1
com.nhn.android.kin                   1
Length: 864, dtype: int64


In [162]:
burger= df1[(df1["subject_id"] == 27) & (df1["name"] == "Burgerking")]

burger

,timestamp,utcoffset,subject_id,source_type,source_info,package_name,name,is_system_app,is_updated_system_app,type
258,2018-10-29 00:48:12,9,27,PHONE,samsung-SM-G930K-8.0.0,kr.co.burgerkinghybrid,Burgerking,0,0,MOVE_TO_FOREGROUND
259,2018-10-29 00:48:12,9,27,PHONE,samsung-SM-G930K-8.0.0,kr.co.burgerkinghybrid,Burgerking,0,0,MOVE_TO_BACKGROUND
260,2018-10-29 00:48:13,9,27,PHONE,samsung-SM-G930K-8.0.0,kr.co.burgerkinghybrid,Burgerking,0,0,MOVE_TO_FOREGROUND
261,2018-10-29 00:48:19,9,27,PHONE,samsung-SM-G930K-8.0.0,kr.co.burgerkinghybrid,Burgerking,0,0,MOVE_TO_BACKGROUND


위처럼 핸드폰에서 앱을 시작하면서 변화되는 모든 순간을 시간으로 기록되는 것을 관찰할 수 있습니다.   
이를 이용하여 사용자가 해당 앱에서 얼마나 머물렸는지를 확인하기 위해 시작되는 시간과 마지막 종료되는 시점을 제외하고 모두 제거하는 작업을 진행토록 하겠습니다. 

In [163]:
df1[(df1["name"] == "토스")]

,timestamp,utcoffset,subject_id,source_type,source_info,package_name,name,is_system_app,is_updated_system_app,type
19723,2018-10-05 11:35:42,9,17,PHONE,samsung-SM-A710S-7.0,viva.republica.toss,토스,0,0,MOVE_TO_FOREGROUND
19724,2018-10-05 11:35:45,9,17,PHONE,samsung-SM-A710S-7.0,viva.republica.toss,토스,0,0,MOVE_TO_BACKGROUND
19725,2018-10-05 11:35:45,9,17,PHONE,samsung-SM-A710S-7.0,viva.republica.toss,토스,0,0,MOVE_TO_FOREGROUND
19726,2018-10-05 11:35:54,9,17,PHONE,samsung-SM-A710S-7.0,viva.republica.toss,토스,0,0,MOVE_TO_BACKGROUND
19736,2018-10-05 11:39:24,9,17,PHONE,samsung-SM-A710S-7.0,viva.republica.toss,토스,0,0,MOVE_TO_FOREGROUND
...,...,...,...,...,...,...,...,...,...,...
1433841,2018-10-17 14:34:38,9,9,PHONE,samsung-SM-N950N-8.0.0,viva.republica.toss,토스,0,0,MOVE_TO_BACKGROUND
1433842,2018-10-17 14:34:38,9,9,PHONE,samsung-SM-N950N-8.0.0,viva.republica.toss,토스,0,0,MOVE_TO_FOREGROUND
1433843,2018-10-17 14:34:41,9,9,PHONE,samsung-SM-N950N-8.0.0,viva.republica.toss,토스,0,0,MOVE_TO_BACKGROUND
1433844,2018-10-17 14:34:41,9,9,PHONE,samsung-SM-N950N-8.0.0,viva.republica.toss,토스,0,0,MOVE_TO_FOREGROUND


In [164]:
df1["timestamp"] = pd.to_datetime(df1["timestamp"])
df1["date"] = df1["timestamp"].map(lambda x: x.date())
df1["time"] = df1["timestamp"].map(lambda x: x.time())
df1_copy = df1

In [165]:
df1_2 = df1_copy.set_index(["subject_id", "date"])
df1_2

timestamp  utcoffset source_type  \
subject_id date                                                    
5          2018-10-29 2018-10-29 05:29:49          9       PHONE   
           2018-10-29 2018-10-29 05:29:54          9       PHONE   
           2018-10-29 2018-10-29 05:29:54          9       PHONE   
           2018-10-29 2018-10-29 05:29:54          9       PHONE   
           2018-10-29 2018-10-29 05:30:01          9       PHONE   
...                                   ...        ...         ...   
29         2018-10-27 2018-10-27 14:11:30          9       PHONE   
           2018-10-27 2018-10-27 14:11:30          9       PHONE   
           2018-10-27 2018-10-27 14:11:33          9       PHONE   
           2018-10-27 2018-10-27 14:11:33          9       PHONE   
           2018-10-27 2018-10-27 14:12:26          9       PHONE   

                                source_info          package_name    name  \
subject_id date                                                             
5          2018-10-29  samsung-SM-G928S-7.0  com.nhn.android.band    BAND   
           2018-10-29  samsung-SM-G928S-7.0  com.nhn.android.band    BAND   
           2018-10-29  samsung-SM-G928S-7.0  com.nhn.android.band    BAND   
           2018-10-29  samsung-SM-G928S-7.0  com.nhn.android.band    BAND   
           2018-10-29  samsung-SM-G928S-7.0  com.nhn.android.band    BAND   
...                                     ...                   ...     ...   
29         2018-10-27    LGE-LG-F700L-8.0.0  com.xiaomi.hm.health   Mi 피트   
           2018-10-27    LGE-LG-F700L-8.0.0     com.lge.launcher3     기본홈   
           2018-10-27    LGE-LG-F700L-8.0.0     com.lge.launcher3     기본홈   
           2018-10-27    LGE-LG-F700L-8.0.0    com.android.chrome  Chrome   
           2018-10-27    LGE-LG-F700L-8.0.0    com.android.chrome  Chrome   

                       is_system_app  is_updated_system_app  \
subject_id date                                               
5          2018-10-29              0                      0   
           2018-10-29              0                      0   
           2018-10-29              0                      0   
           2018-10-29              0                      0   
           2018-10-29              0                      0   
...                              ...                    ...   
29         2018-10-27              0                      0   
           2018-10-27              1                      0   
           2018-10-27              1                      0   
           2018-10-27              1                      1   
           2018-10-27              1                      1   

                                     type      time  
subject_id date                                      
5          2018-10-29    USER_INTERACTION  05:29:49  
           2018-10-29  MOVE_TO_FOREGROUND  05:29:54  
           2018-10-29  MOVE_TO_BACKGROUND  05:29:54  
           2018-10-29  MOVE_TO_FOREGROUND  05:29:54  
           2018-10-29  MOVE_TO_BACKGROUND  05:30:01  
...                                   ...       ...  
29         2018-10-27    USER_INTERACTION  14:11:30  
           2018-10-27  MOVE_TO_FOREGROUND  14:11:30  
           2018-10-27  MOVE_TO_BACKGROUND  14:11:33  
           2018-10-27  MOVE_TO_FOREGROUND  14:11:33  
           2018-10-27  MOVE_TO_BACKGROUND  14:12:26  

[1588338 rows x 10 columns]

In [166]:
stay_time = []
for i in range(len(df1_2)):
  now = df1_2.iloc[i]
  if i != len(df1_2) - 1:
    next = df1_2.iloc[i+1]
    stay_time.append(next['timestamp'] - now['timestamp'])
  elif i == len(df1_2) - 1:
    stay_time.append(None) 

In [167]:
df1_2['stay_time'] = stay_time

In [168]:
df1_2[1:2]

,,timestamp,utcoffset,source_type,source_info,package_name,name,is_system_app,is_updated_system_app,type,time,stay_time
subject_id,date,,,,,,,,,,,
5,2018-10-29,2018-10-29 05:29:54,9,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND,05:29:54,0 days


In [169]:
df1_2["all_time"] = 0

In [183]:
k = 0
j = 0
all = [] 
for i in range(len(df1_copy)):
  now = df1_2.iloc[k]
  next = df1_2.iloc[i]

  if now["name"] == next["name"]:
    j = j + 1
    all.append(0)
  else :
    time = next["timestamp"] - now["timestamp"]
    all.append(time)
    k = k + j
    j = 0

df1_2["all_time"] = all

In [184]:
df1_2[:20]

timestamp  utcoffset source_type  \
subject_id date                                                    
5          2018-10-29 2018-10-29 05:29:49          9       PHONE   
           2018-10-29 2018-10-29 05:29:54          9       PHONE   
           2018-10-29 2018-10-29 05:29:54          9       PHONE   
           2018-10-29 2018-10-29 05:29:54          9       PHONE   
           2018-10-29 2018-10-29 05:30:01          9       PHONE   
           2018-10-29 2018-10-29 05:30:01          9       PHONE   
           2018-10-29 2018-10-29 05:30:07          9       PHONE   
           2018-10-29 2018-10-29 05:30:07          9       PHONE   
           2018-10-29 2018-10-29 05:30:22          9       PHONE   
           2018-10-29 2018-10-29 05:30:22          9       PHONE   
           2018-10-29 2018-10-29 05:30:31          9       PHONE   
           2018-10-29 2018-10-29 05:30:31          9       PHONE   
           2018-10-29 2018-10-29 05:30:38          9       PHONE   
           2018-10-29 2018-10-29 05:30:38          9       PHONE   
           2018-10-29 2018-10-29 05:30:40          9       PHONE   
           2018-10-29 2018-10-29 05:30:40          9       PHONE   
           2018-10-29 2018-10-29 05:30:43          9       PHONE   
           2018-10-29 2018-10-29 05:30:43          9       PHONE   
           2018-10-29 2018-10-29 05:30:45          9       PHONE   
           2018-10-29 2018-10-29 05:30:45          9       PHONE   

                                source_info                  package_name  \
subject_id date                                                             
5          2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0          com.nhn.android.band   
           2018-10-29  samsung-SM-G928S-7.0  com.samsung.android.contacts   
           2018-10-29  samsung-SM-G928S-7.0  com.samsung.android.contacts   
           2018-10-29  samsung-SM-G928S-7.0  com.sec.android.app.launcher   
           2018-10-29  samsung-SM-G928S-7.0  com.sec.android.app.launcher   
           2018-10-29  samsung-SM-G928S-7.0  com.sec.android.app.sbrowser   

                             name  is_system_app  is_updated_system_app  \
subject_id date                                                           
5          2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018-10-29        BAND              0                      0   
           2018

In [ ]:
# 데이터 pickle로 저장 & 전처리 시간이 매우 오래걸리기 때문에 해당 데이터를 저장해두도록 하겠습니다. 
df1_2.to_csv("토스분석.csv", index=False)

In [10]:
ls

스마트폰_로그_데이터셋2.json  토스분석.csv  스마트폰_로그_데이터셋.json


In [12]:
# 저장이 정상적으로 되었는 확인하겠습니다. 
df1 = pd.read_csv("토스분석.csv")

In [13]:
df1.shape

(1588338, 12)

In [14]:
df1.head()

,timestamp,utcoffset,source_type,source_info,package_name,name,is_system_app,is_updated_system_app,type,time,stay_time,all_time
0,2018-10-29 05:29:49,9,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,USER_INTERACTION,05:29:49,0 days 00:00:05,0
1,2018-10-29 05:29:54,9,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND,05:29:54,0 days 00:00:00,0
2,2018-10-29 05:29:54,9,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_BACKGROUND,05:29:54,0 days 00:00:00,0
3,2018-10-29 05:29:54,9,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_FOREGROUND,05:29:54,0 days 00:00:07,0
4,2018-10-29 05:30:01,9,PHONE,samsung-SM-G928S-7.0,com.nhn.android.band,BAND,0,0,MOVE_TO_BACKGROUND,05:30:01,0 days 00:00:00,0


In [15]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 7.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
app_list = df1.name.unique()

from selenium import webdriver
from time import sleep
from IPython.display import clear_output

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.implicitly_wait(2)


genre_dict = {}
error_list = []

for i, key_word in enumerate(app_list):
    driver.get('https://play.google.com/store/apps')

    try:
        clear_output()
        print(f'{i+1}/833') #823 = len(app_list)
        driver.find_element_by_xpath('//*[@id="gbqfq"]').send_keys(f'{key_word}' + '\n') #\n을 같이 써줬기에 페이지로 한번에 이동함
        selected_tag_a = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/c-wiz/c-wiz/c-wiz/div/div[2]/div[1]/c-wiz/div/div/div[1]/div/div/a') #find_element_by_css_selector("input[tabindex='-1']") #
        selected_tag_a.click()
        elements = driver.find_elements_by_css_selector('a[itemprop="genre"]')
        genre_dict[key_word] = elements[0].text

    except:
        clear_output()
        print(f'{i}/822')
        error_list.append(key_word)

In [ ]:
# 수집된 장르 확인 
print("수집된 정상장르 : ", genre_dict)
print("---")
print("수집된 에러장르 : ", error_list)

In [ ]:
set(genre_dict.values())

In [231]:
# 에러로 빠져서 수집이 되지 않은 경우 직접 genre_dict에 넣어준다. 
genre_dict['KB앱카드'] = 'Finance'
genre_dict['OP.GG'] = 'Strategy'
genre_dict['커피빈멤버스클럽'] = 'Lifestyle'
genre_dict['퀵툴'] = 'Lifestyle'
genre_dict['기본홈(홈&앱서랍)'] = 'HOME'
genre_dict['爱奇艺HD'] = 'Entertainment'
genre_dict['GrimTalk'] = 'Puzzle'
genre_dict['U+프로야구'] = 'Sports'
genre_dict['TubeMate'] = 'Video Players & Editors'

In [ ]:
# #정리가 편하도록 대장르를 정리한 뒤 key, value값을 swap해서 매핑

# genre_bigcol = {
#     'social' : ['Social','Dating'],
#     'hobby' : ['Books & Reference','Art & Design','Entertainment','Video', 'Video Players & Editors',
#              'Shopping','Sports','Comics','Photography','Beauty'],
#     'self development' : ['Fitness', 'Health & Fitness', 'Education',"Educational", 'Business'],
#     'contact' : ['Communication','contact'],
#     'tools' : ['Personalization','Tools','HOME','tools'],
#     'lifestype' : ['Music & Audio','News & Magazines','Food & Drink','Lifestyle','lifestype'],
#     'producivity' : ['Productivity','House & Home','Auto & Vehicles','Libraries & Demo','Weather',
#                      'Travel & Local','Maps & Navigation', 'Finance'],
#     'game' : ['Role Playing', 'Simulation', 'Puzzle', 'Casual', 'Adventure',
#            'Racing', 'Action', 'Strategy', 'Parenting', 'Arcade','Board']
# }

# genre_bigcol = {v:k for k,v_list in genre_bigcol.items() for v in v_list}
# genre_bigcol

In [232]:
# 장르 커스트마이징 
genre_bigcol = {
    'social' : ['Social','Dating', 'Communication','contact'],
    'hobby' : ['Books & Reference','Art & Design','Entertainment',
             'Shopping','Comics','Photography','Beauty', 'Music & Audio','News & Magazines','Food & Drink','Lifestyle','lifestype'],
    'self development' : ['Fitness', 'Health & Fitness', 'Education',"Educational", 'Business'],
    'producivity' : ['Productivity','House & Home','Auto & Vehicles','Libraries & Demo','Weather',
                     'Travel & Local','Maps & Navigation', 'Finance', 'Personalization','Tools','HOME','tools'],
    'game' : ['Sports', 'Video', 'Video Players & Editors', 'Role Playing', 'Simulation', 'Puzzle', 'Casual', 'Adventure', 'Racing', 'Action', 'Strategy', 'Parenting', 'Arcade','Board']
}

genre_bigcol = {v:k for k,v_list in genre_bigcol.items() for v in v_list}
genre_bigcol

{'Action': 'game',
 'Adventure': 'game',
 'Arcade': 'game',
 'Art & Design': 'hobby',
 'Auto & Vehicles': 'producivity',
 'Beauty': 'hobby',
 'Board': 'game',
 'Books & Reference': 'hobby',
 'Business': 'self development',
 'Casual': 'game',
 'Comics': 'hobby',
 'Communication': 'social',
 'Dating': 'social',
 'Education': 'self development',
 'Educational': 'self development',
 'Entertainment': 'hobby',
 'Finance': 'producivity',
 'Fitness': 'self development',
 'Food & Drink': 'hobby',
 'HOME': 'producivity',
 'Health & Fitness': 'self development',
 'House & Home': 'producivity',
 'Libraries & Demo': 'producivity',
 'Lifestyle': 'hobby',
 'Maps & Navigation': 'producivity',
 'Music & Audio': 'hobby',
 'News & Magazines': 'hobby',
 'Parenting': 'game',
 'Personalization': 'producivity',
 'Photography': 'hobby',
 'Productivity': 'producivity',
 'Puzzle': 'game',
 'Racing': 'game',
 'Role Playing': 'game',
 'Shopping': 'hobby',
 'Simulation': 'game',
 'Social': 'social',
 'Sports': 'ga

In [ ]:
# 장르 딕셔너리를 바탕으로 어플마다 장르를 매칭
df1['genre'] = df1['name'].map(lambda x: genre_dict[x]) 

In [ ]:
df1['genre_bigcol'] = df1['name'].map(lambda x: genre_bigcol[x]) #장르 딕셔너리를 바탕으로 어플마다 장르를 매칭

In [ ]:
# 사용자가 특정 어플리케이션으로 이동하기 전에 사용한 어플리케이션의 이름 장르 패키지를 보여주는 특성 
df1['referrer_package'] = [None] + list(df1['package_name'])[:-1]
df1['referrer_app'] = [None] + list(df1['name'])[:-1]
df1['referrer_genre'] = [None] + list(df1['genre'])[:-1]

In [ ]:
df1

In [ ]:
df1.head(2)

In [ ]:
df1.shape